### Fixed simulation configuration

In [1]:
import sys
sys.path.append(r"D:\Papers\4) Finished Articles\6. MWSN - DCHPC\DCHPC")

In [2]:
import os
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm

from simulation import Simulation

In [3]:
OUTPUT_DIR = r"D:\Papers\4) Finished Articles\6. MWSN - DCHPC\DCHPC\AParameters_tuning"

In [4]:
BASE_CONFIG = dict(
    area_size=(100, 100),
    n_nodes=100,
    rounds=60000,
    init_energy=0.5,
    comm_range=50.0,
    sink_mode="adaptive",
    routing_mode="multi-hop",
    localization_mode="random",
    seed=42,
    head_selection_strategy="optimizer"
)

In [5]:
# run single sim and extract metrics
def run_sim_get_metrics(extra_config):
    try:
        sim = Simulation(**BASE_CONFIG, **extra_config)
        metrics = sim.run()
        return {
            'LND': metrics.get('LND', 0),
            'PDR': metrics.get('PDR', 0),
            'Avg_E2E_Delay_Sec': metrics.get('Avg_E2E_Delay_Sec', 0)
        }
    except Exception as e:
        print(f"Failed: {e}")
        return {'LND': 0, 'PDR': 0, 'Avg_E2E_Delay_Sec': float('inf')}

### Routing Weights Tuning

In [6]:
# Weight grid: we fix sum = 1.0
# Focus on reasonable ranges based on intuition
distance_vals = [0.4, 0.5, 0.6]
energy_vals   = [0.2, 0.3, 0.4]
load_vals     = [0.05, 0.1, 0.15]
trust_vals    = [0.05, 0.1, 0.15]

In [10]:
def valid_combo(wd, we, wl, wt):
    return abs(wd + we + wl + wt - 1.0) < 1e-6

# Generate valid combinations
combinations = []
for wd, we, wl, wt in itertools.product(distance_vals, energy_vals, load_vals, trust_vals):
    if valid_combo(wd, we, wl, wt):
        combinations.append((wd, we, wl, wt))
print(f"Testing {len(combinations)} routing weight combinations.")

Testing 13 routing weight combinations.


In [11]:
routing_results = []
for weights in tqdm(combinations, desc="Routing"):
    wd, we, wl, wt = weights
    metrics = run_sim_get_metrics({
        'weight_distance': wd,
        'weight_energy': we,
        'weight_load': wl,
        'weight_trust': wt
    })
    routing_results.append({
        'param_group': 'routing',
        'weight_distance': wd,
        'weight_energy': we,
        'weight_load': wl,
        'weight_trust': wt,
        **metrics
    })

Routing:   0%|          | 0/13 [00:00<?, ?it/s]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Routing:   8%|▊         | 1/13 [00:29<05:50, 29.19s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Routing:  15%|█▌        | 2/13 [01:07<06:17, 34.31s/it]

...

Routing:  85%|████████▍ | 11/13 [06:21<01:06, 33.40s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Routing:  92%|█████████▏| 12/13 [06:50<00:31, 31.88s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Routing: 100%|██████████| 13/13 [07:22<00:00, 34.03s/it]


In [12]:
pd.DataFrame(routing_results).to_csv(os.path.join(OUTPUT_DIR, "routing.csv"), index=False)

In [ ]:
df_r = pd.DataFrame(routing_results)
best_r = df_r.loc[df_r['LND'].idxmax()]
print(f"Routing: d={best_r['weight_distance']}, e={best_r['weight_energy']}, "
      f"l={best_r['weight_load']}, t={best_r['weight_trust']} → LND={best_r['LND']:.0f}")

Routing: d=0.5, e=0.2, l=0.15, t=0.15 → LND=16031


In [22]:
df_r.head(5).sort_values('LND', ascending=False)

,param_group,weight_distance,weight_energy,weight_load,weight_trust,LND,PDR,Avg_E2E_Delay_Sec
4,routing,0.5,0.2,0.15,0.15,16031,0.998652,14.901961
2,routing,0.4,0.4,0.10,0.10,9471,0.998532,7.767766
0,routing,0.4,0.3,0.15,0.15,4566,0.998709,6.949919
3,routing,0.4,0.4,0.15,0.05,3671,0.999436,10.513931
1,routing,0.4,0.4,0.05,0.15,3099,0.999350,6.100733


### Reclustering Parameters Tuning

In [6]:
reclust_defaults = {
    'recluster_period': 50,
    'energy_threshold': 0.3,
    'load_threshold': 10,
    'sink_move_threshold': 20.0
}
reclust_tests = {
    'recluster_period': [10, 20, 30, 50, 75, 100],
    'energy_threshold': [0.1, 0.2, 0.3, 0.4, 0.5],
    'load_threshold': [5, 10, 15, 20, 25],
    'sink_move_threshold': [10, 15, 20, 25, 30]
}

In [7]:
reclust_results = []
for param, values in reclust_tests.items():
    for val in tqdm(values, desc=f"Reclust: {param}"):
        config = {k: v for k, v in reclust_defaults.items() if k != param}
        config[param] = val
        metrics = run_sim_get_metrics(config)
        reclust_results.append({
            'param_group': 'reclustering',
            'tested_param': param,
            'tested_value': val,
            **metrics
        })

Reclust: recluster_period:   0%|          | 0/6 [00:00<?, ?it/s]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Reclust: recluster_period:  17%|█▋        | 1/6 [00:44<03:40, 44.02s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Reclust: recluster_period:  33%|███▎      | 2/6 [01:24<02:47, 41.81s/it]

...

Reclust: sink_move_threshold:  60%|██████    | 3/5 [01:39<01:06, 33.19s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Reclust: sink_move_threshold:  80%|████████  | 4/5 [02:12<00:33, 33.05s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Reclust: sink_move_threshold: 100%|██████████| 5/5 [02:45<00:00, 33.13s/it]


In [8]:
pd.DataFrame(reclust_results).to_csv(os.path.join(OUTPUT_DIR, "reclustering_params.csv"), index=False)

In [ ]:
test = []
df_rc = pd.DataFrame(reclust_results)
for param in reclust_tests.keys():
    subset = df_rc[df_rc['tested_param'] == param]
    test.append(subset.sort_values(by='LND', ascending=False))
    best = subset.loc[subset['LND'].idxmax()]
    print(f"Reclust {param}: {best['tested_value']} → LND={best['LND']:.0f}")

Reclust recluster_period: 75.0 → LND=15160
Reclust energy_threshold: 0.1 → LND=8956
Reclust load_threshold: 10.0 → LND=8956
Reclust sink_move_threshold: 10.0 → LND=8956


In [22]:
test[0].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
4,reclustering,recluster_period,75.0,15160,0.998441,5.231432
3,reclustering,recluster_period,50.0,8956,0.998889,6.645042
0,reclustering,recluster_period,10.0,4068,0.999315,14.242857


In [23]:
test[1].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
6,reclustering,energy_threshold,0.1,8956,0.998889,6.645042
7,reclustering,energy_threshold,0.2,8956,0.998889,6.645042
8,reclustering,energy_threshold,0.3,8956,0.998889,6.645042


In [24]:
test[2].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
12,reclustering,load_threshold,10.0,8956,0.998889,6.645042
14,reclustering,load_threshold,20.0,3539,0.999065,3.918143
13,reclustering,load_threshold,15.0,3539,0.999065,3.918143


In [25]:
test[3].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
16,reclustering,sink_move_threshold,10.0,8956,0.998889,6.645042
17,reclustering,sink_move_threshold,15.0,8956,0.998889,6.645042
18,reclustering,sink_move_threshold,20.0,8956,0.998889,6.645042


### Gravitational Optimizer Parameters Tuning

In [6]:
go_tests = {
    'go_iterations': [5, 10, 15, 20, 25],
    'population_size': [5, 10, 15, 20],
    'alpha': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    'G0': [50, 100, 150, 200]
    # beta = 1 - alpha, so not tested independently
}

In [7]:
go_results = []
for param, values in go_tests.items():
    for val in tqdm(values, desc=f"GO: {param}"):
        if param == 'alpha':
            # Auto-set beta = 1 - alpha
            metrics = run_sim_get_metrics({'alpha': val, 'beta': 1.0 - val})
        else:
            metrics = run_sim_get_metrics({param: val})
        go_results.append({
            'param_group': 'optimizer',
            'tested_param': param,
            'tested_value': val,
            **metrics
        })


GO: go_iterations:   0%|          | 0/5 [00:00<?, ?it/s]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


GO: go_iterations:  20%|██        | 1/5 [00:18<01:15, 18.98s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


GO: go_iterations:  40%|████      | 2/5 [00:48<01:15, 25.10s/it]

...

GO: G0:  50%|█████     | 2/4 [01:16<01:16, 38.30s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


GO: G0:  75%|███████▌  | 3/4 [01:54<00:38, 38.11s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


GO: G0: 100%|██████████| 4/4 [02:33<00:00, 38.25s/it]


In [8]:
pd.DataFrame(go_results).to_csv(os.path.join(OUTPUT_DIR, "gravitational_params.csv"), index=False)

In [12]:
test = []
df_rc = pd.DataFrame(go_results)
for param in go_tests.keys():
    subset = df_rc[df_rc['tested_param'] == param]
    test.append(subset.sort_values(by='LND', ascending=False))
    best = subset.loc[subset['LND'].idxmax()]
    print(f"OGS {param}: {best['tested_value']} → LND={best['LND']:.0f}")

OGS go_iterations: 15.0 → LND=15160
OGS population_size: 10.0 → LND=15160
OGS alpha: 0.6 → LND=15160
OGS G0: 50.0 → LND=15160


In [14]:
test[0].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
2,optimizer,go_iterations,15.0,15160,0.998441,5.231432
3,optimizer,go_iterations,20.0,13100,0.998297,5.234064
1,optimizer,go_iterations,10.0,6770,0.998450,11.942616


In [15]:
test[1].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
6,optimizer,population_size,10.0,15160,0.998441,5.231432
8,optimizer,population_size,20.0,2811,0.998914,7.859564
7,optimizer,population_size,15.0,1758,0.998859,8.360843


In [16]:
test[2].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
12,optimizer,alpha,0.6,15160,0.998441,5.231432
9,optimizer,alpha,0.3,5163,0.998958,5.989695
14,optimizer,alpha,0.8,3707,0.998884,8.312124


In [17]:
test[3].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
15,optimizer,G0,50.0,15160,0.998441,5.231432
16,optimizer,G0,100.0,15160,0.998441,5.231432
17,optimizer,G0,150.0,15160,0.998441,5.231432


### Mobile Sink Parameters Tuning

In [6]:
sink_tests = {
    'visit_period': [2, 3, 5, 7, 10],
    'energy_weight': [0.4, 0.5, 0.6, 0.7, 0.8],
    # distance_weight = 1 - energy_weight, so not tested independently
}

In [7]:
sink_results = []
for param, values in sink_tests.items():
    for val in tqdm(values, desc=f"Sink: {param}"):
        if param == 'energy_weight':
            # Auto-set distance_weight = 1 - energy_weight
            metrics = run_sim_get_metrics({'distance_weight': val, 'energy_weight': 1.0 - val})
        else:
            metrics = run_sim_get_metrics({param: val})
        sink_results.append({
            'param_group': 'sink',
            'tested_param': param,
            'tested_value': val,
            **metrics
        })

Sink: visit_period:   0%|          | 0/5 [00:00<?, ?it/s]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Sink: visit_period:  20%|██        | 1/5 [00:33<02:12, 33.24s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Sink: visit_period:  40%|████      | 2/5 [01:04<01:35, 31.88s/it]

...

Sink: energy_weight:  60%|██████    | 3/5 [01:45<01:10, 35.20s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Sink: energy_weight:  80%|████████  | 4/5 [02:20<00:35, 35.15s/it]

[Info] Sink speed: 25.0 m/round = 0.50 m/s


Sink: energy_weight: 100%|██████████| 5/5 [02:55<00:00, 35.15s/it]


In [8]:
pd.DataFrame(sink_results).to_csv(os.path.join(OUTPUT_DIR, "mobilesink_params.csv"), index=False)

In [9]:
test = []
df_rc = pd.DataFrame(sink_results)
for param in sink_tests.keys():
    subset = df_rc[df_rc['tested_param'] == param]
    test.append(subset.sort_values(by='LND', ascending=False))
    best = subset.loc[subset['LND'].idxmax()]
    print(f"OGS {param}: {best['tested_value']} → LND={best['LND']:.0f}")

OGS visit_period: 5.0 → LND=15160
OGS energy_weight: 0.4 → LND=15160


In [10]:
test[0].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
2,sink,visit_period,5.0,15160,0.998441,5.231432
1,sink,visit_period,3.0,14327,0.998368,10.143378
0,sink,visit_period,2.0,4691,0.999131,10.829193


In [11]:
test[1].head(3)

,param_group,tested_param,tested_value,LND,PDR,Avg_E2E_Delay_Sec
5,sink,energy_weight,0.4,15160,0.998441,5.231432
6,sink,energy_weight,0.5,15160,0.998441,5.231432
7,sink,energy_weight,0.6,15160,0.998441,5.231432
